# An example of neutrino propagation simulation through flexOPT.jl 
Some of functions are the fruit of works from Estelle Salomé (July-August 2025) and TIPE (2025-2026).

In [ ]:

using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using Colors



  Activating 

data directory which is local:

In [ ]:
dir="/Users/nobuaki/Documents/MantleConvectionTakashi/op_old_full_mars_2025/"

## making 2D(/3D) box with the aid of GeoPoint.jl (not necessary for geodynamic models)

Please refer to getRegionalBox.ipynb if you have some questions 

In [ ]:
set_default_planet!(:SphericalEarth) # SphericalEarth means a perfect sphere with a radius of 6371000.0 m 
# otherwise of course, I made an Earth with ellipticity

In [ ]:
# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(0.0,0.0;alt=30.e3) # lat, lon in degrees, altitude in metre (as an option)
# if you call GeoPoints with three floats without ; for alt, it will interpret as x, y, z in cartesian ecef coordinates
p2 = GeoPoint(0.0,180.0;alt=30.e3) # 


Δx = 25.e3 # in metre
Δz = 25.e3

altMax = 30.e3 # in metre
altMin = -(6371.e3*2)-30.e3 # in metre

In [ ]:
boxGrids=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax;centreOption ="centreOfPlanet")

In [ ]:
# getting model parameters on the 2D disk
seismicModel=lazyProduceOrLoad("seismicModel2DGlobal",getParamsWithoutTopo,boxGrids.allGridsInGeoPoints,boxGrids.effectiveRadii,2.0)

In [ ]:
Nx,Nz=size(seismicModel.ρ)

# Reading models from Takashi and Paul

Paul is ready to give us more models whenever we want but the data are confidential and I cannot share on the Github

In [ ]:

rhoFiles=myListDir(dir; pattern=r"test_rho\d");
compositionFiles=myListDir(dir; pattern=r"test_c\d");
temperatureFiles=myListDir(dir; pattern=r"test_t\d");
wtrFiles=myListDir(dir; pattern=r"test_wtr\d");